# Project 3
## Tyler Baker and Rathish Parayil Sasidharan

The goal of this project is to take the names dataset and to genderclassify each name. We will have test groups so that we can compare our results.

### Setting up the Environment

In [24]:
from nltk.corpus import names
import nltk
import numpy as np
import pandas as pd
import random

### The Data

In [14]:
names.words()

['Abagael',
 'Abagail',
 'Abbe',
 'Abbey',
 'Abbi',
 'Abbie',
 'Abby',
 'Abigael',
 'Abigail',
 'Abigale',
 'Abra',
 'Acacia',
 'Ada',
 'Adah',
 'Adaline',
 'Adara',
 'Addie',
 'Addis',
 'Adel',
 'Adela',
 'Adelaide',
 'Adele',
 'Adelice',
 'Adelina',
 'Adelind',
 'Adeline',
 'Adella',
 'Adelle',
 'Adena',
 'Adey',
 'Adi',
 'Adiana',
 'Adina',
 'Adora',
 'Adore',
 'Adoree',
 'Adorne',
 'Adrea',
 'Adria',
 'Adriaens',
 'Adrian',
 'Adriana',
 'Adriane',
 'Adrianna',
 'Adrianne',
 'Adrien',
 'Adriena',
 'Adrienne',
 'Aeriel',
 'Aeriela',
 'Aeriell',
 'Ag',
 'Agace',
 'Agata',
 'Agatha',
 'Agathe',
 'Aggi',
 'Aggie',
 'Aggy',
 'Agna',
 'Agnella',
 'Agnes',
 'Agnese',
 'Agnesse',
 'Agneta',
 'Agnola',
 'Agretha',
 'Aida',
 'Aidan',
 'Aigneis',
 'Aila',
 'Aile',
 'Ailee',
 'Aileen',
 'Ailene',
 'Ailey',
 'Aili',
 'Ailina',
 'Ailyn',
 'Aime',
 'Aimee',
 'Aimil',
 'Aina',
 'Aindrea',
 'Ainslee',
 'Ainsley',
 'Ainslie',
 'Ajay',
 'Alaine',
 'Alameda',
 'Alana',
 'Alanah',
 'Alane',
 'Alanna',
 

In [16]:
len(names.words())

7944

We will want to split the 7944 into three groups.

test_set will have 500 words.

dev_test_set will have 500 words.

training_set will have 6944.

In [18]:
names.fileids()

['female.txt', 'male.txt']

In [46]:
males = [(name, "male") for name in names.words("male.txt")]
females = [(name, "female") for name in names.words("female.txt")]
corpus_list = males+females

### Creating the Simulation

In [63]:
def accuracy(number_of_runs, function):
    accuracy_df = {
        "classifier": [],
        "train_set_acc": [],
        "dev_set_acc": [],
        "test_acc": []
    }
    for i in range(number_of_runs):
        random.shuffle(corpus_list)
        dev_name_set = corpus_list[0:500]
        test_name_set = corpus_list[500:1000]
        train_name_set = corpus_list[1000:]
        dev_set = [(function(n), i) for (n, i) in dev_name_set]
        test_set = [(function(n), i) for (n, i) in test_name_set]
        train_set = [(function(n), i) for (n, i) in train_name_set]
        classifier = nltk.NaiveBayesClassifier.train(train_set)
        accuracy_df["classifier"].append(classifier)
        accuracy_df["train_set_acc"].append(nltk.classify.accuracy(classifier, train_set))
        accuracy_df["dev_set_acc"].append(nltk.classify.accuracy(classifier, dev_set))
        accuracy_df["test_acc"].append(nltk.classify.accuracy(classifier, test_set))
    accuracy_df = pd.DataFrame.from_dict(accuracy_df)
    return(accuracy_df)

This is the first feature shown in the text book. It simply classifies based on the last letter of the first name.

### Creating Features

In [64]:
def last_letter(name):
    return {"last_letter": name[-1]}

We were curious if we could just classify based on the length of a name.

In [65]:
def length_of_name(name):
    return {"length": len(name)}

In [67]:
last_letter_df = accuracy(100, last_letter)

In [68]:
last_letter_df.describe()

,train_set_acc,dev_set_acc,test_acc
count,100.000000,100.000000,100.000000
mean,0.762800,0.763300,0.760460
std,0.001736,0.017724,0.017608
min,0.758641,0.712000,0.712000
25%,0.761485,0.752000,0.750000
50%,0.762673,0.763000,0.760000
75%,0.763969,0.776000,0.774000
max,0.766993,0.808000,0.808000


In [69]:
length_df = accuracy(100, length_of_name)

In [70]:
length_df.describe()

,train_set_acc,dev_set_acc,test_acc
count,100.000000,100.000000,100.00000
mean,0.632559,0.631560,0.63492
std,0.002115,0.021459,0.02195
min,0.626872,0.576000,0.58200
25%,0.631048,0.619500,0.62000
50%,0.632416,0.630000,0.63300
75%,0.633965,0.648500,0.65000
max,0.638681,0.680000,0.69000


Even though are results were not what we were hoping for, 63% is still pretty high for just the length of a name. Combing length of name, and end letters would be an interesting attempt.

In [71]:
def combined_features(name):
    return{"last_letter": name[-1], "length": len(name)}

In [72]:
combined_df = accuracy(100, combined_features)

In [73]:
combined_df.describe()

,train_set_acc,dev_set_acc,test_acc
count,100.000000,100.000000,100.000000
mean,0.763949,0.761060,0.761080
std,0.001886,0.019763,0.018665
min,0.759073,0.710000,0.718000
25%,0.762925,0.748000,0.748000
50%,0.764113,0.762000,0.761000
75%,0.765157,0.776500,0.770500
max,0.769153,0.810000,0.804000


Combining the two features has only made a slight increase. We decided to add more features. 

We will add the feature of the first letter as well.

In [74]:
def first_last_len(name):
    return{"first_letter":name[0], "last_letter":name[-1], "length":len(name)}

In [75]:
fsl_df = accuracy(100, first_last_len)

In [76]:
fsl_df.describe()

,train_set_acc,dev_set_acc,test_acc
count,100.000000,100.000000,100.000000
mean,0.779201,0.776000,0.776320
std,0.001932,0.018381,0.017917
min,0.773474,0.720000,0.738000
25%,0.777902,0.761500,0.764000
50%,0.779306,0.777000,0.776000
75%,0.780134,0.790500,0.790500
max,0.785282,0.820000,0.834000


Using just the first letter, last letter, and the length of the name we are able to on average predict the gender of a name correctly about 78% of the time.

Finally, We would like to try to do this for the first two letters, the last two letters, and the length.

In [77]:
def final_feat(name):
    return{"first_letters":name[:2], "last_letters":name[-2:], "length":len(name)}

In [78]:
final_df = accuracy(100, final_feat)

In [79]:
final_df.describe()

,train_set_acc,dev_set_acc,test_acc
count,100.000000,100.000000,100.000000
mean,0.830027,0.809340,0.805520
std,0.002078,0.015776,0.016527
min,0.825029,0.776000,0.752000
25%,0.828629,0.799500,0.796000
50%,0.830141,0.808000,0.804000
75%,0.831149,0.818500,0.818000
max,0.836118,0.846000,0.866000


### Conclusion

In conclusion, the more features we added the higher the accuracy. However, there is a statement worth noting. There are many four letter names. When we chose to use the first two, and the last two letters of the name we were guranteed to have some names exactly done. This isn't really prediction at this point.